In [3]:
from wmyblog_coron import genHTML
import pandas as pd

In [2]:
updateBlogPage(days=30)

index page generated
latest page generated
search data generated


In [8]:
df_article = pd.read_pickle('./data/article_full.pkl')
df_comment = pd.read_pickle('./data/comment_full.pkl')
genHTML('171980706',df_article,df_comment)

In [2]:
import pdfkit
pdfkit.from_file('html/108908611.html','test.pdf')

OSError: No wkhtmltopdf executable found: "b''"
If this file exists please check that this process can read it or you can pass path to it manually in method call, check README. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf